In [81]:
from lib.v1_mysql_op import lang_lib
from lib.v1_common import mysql_op
import time
import sys, os
import datetime as dt
import pymysql as db
import pandas as pd
pd.set_option('display.max_columns', 500)

In [82]:
dw = lang_lib(db_name='new_dw')
cur_dw = dw.conn.cursor(db.cursors.DictCursor)

old_dw = lang_lib(db_name='dw')
cur_old_dw = old_dw.conn.cursor(db.cursors.DictCursor) # 加入這段可以匯入表頭

In [83]:
# sql = f"""
# select *
# from lang_share.duration_log_from_prod_2019
# limit 10
# """

sql = f"""
select *
from lang_report.users
limit 10;
"""

In [84]:
cur_old_dw.execute(sql)
data = pd.DataFrame(cur_old_dw.fetchall())

data



# sql = f"""
# ## 近 7 日的觀看時長
# SELECT pfid, SUM(ts) AS watch_duration
# FROM lang_share.duration_log_from_prod
# WHERE LEFT(reg_time, 10) >= CURDATE()-INTERVAL 7 DAY # 近 7 日的觀看紀錄
# #AND LENGTH(pfid)=7
# AND pfid='4758621'
# GROUP BY pfid
# ;
# """

# cur_dw.execute(sql)
# data = pd.DataFrame(cur_dw.fetchall())

# print(data)

,id,email,password,updatedAt,createdAt,role_id,name
0,1,hengyi.lin@langlive.com,$2a$10$4mvgyCO6YrAY8WFxR96uPO1M/cAaAAGy.f.XH52...,2017-06-20 12:24:20,2017-05-31 01:28:22,2,None
1,2,wendell.huang@langlive.com,$2a$10$2HVv5N4sv3eidRKoSC2l3OK5rLY4azn8l/FnC4n...,2018-03-14 09:55:44,2017-05-31 02:02:30,2,None
2,3,jc.wang@langlive.com,$2a$10$sASu61i2zINWGiloFWPQ8.RtBDzysnE87OIczzw...,2020-04-10 10:50:20,2017-05-31 04:03:30,2,None
3,7,rick@test.com,$2a$10$OGSnu.VxIVoetke34kVhJu//Tk8NKl52FrNbiQ8...,2017-06-20 12:24:23,2017-06-20 03:43:09,3,None
4,12,jc123@gmail.com,$2a$10$sASu61i2zINWGiloFWPQ8.RtBDzysnE87OIczzw...,2018-08-03 06:06:48,2017-06-22 00:57:51,12,None
5,13,yiqiao.xu@langlive.com,$2a$10$/5dzU47IGUN9xmpdhicosu2ush7p68sGqigytz8...,2017-06-22 09:22:17,2017-06-22 01:13:29,3,None
6,14,Rick_Lin,,2017-06-26 06:35:58,2017-06-23 08:16:54,2,Rick_Lin
7,15,jcwang,,2018-04-12 13:57:35,2017-06-23 08:37:55,2,jcwang
8,16,sonic.guo@langlive.com,$2a$10$52G24CyHUyooeinArHb9SuYKillVQNjrfEdGODf...,2019-09-11 02:51:19,2017-06-23 03:03:27,0,None
9,17,xi.chen@langlive.com,$2a$10$kZHoA2Mh3xuEQOUR.NkjpuyUaCzCKfXnenvtuO....,2017-06-23 11:10:25,2017-06-23 03:06:02,4,None


In [90]:
def get_duration_data(start, end):
    # 改變 start, end 符合資料庫中的格式
    try:
        start = dt.datetime.strptime(str(start), '%Y%m%d')
        end = dt.datetime.strptime(str(end), '%Y%m%d')
    except Exception as e:
        print('Please Enter start datet and end date by %Y%m%d')
        
    result = pd.DataFrame()
    while start <= end:
        sql = f"""
        select pfid, ts
        from lang_share.duration_log_from_prod
        where reg_time between '{start.strftime('%Y-%m-%d 00:00:00')}' and '{end.strftime('%Y-%m-%d 23:59:59')}'
        """
    
        cur_dw.execute(sql)
        data = pd.DataFrame(cur_dw.fetchall())  
        
        result = result.append(data)
    
        start = start + dt.timedelta(days=1)
        
    return result

In [92]:
s = time.time()
data_7day = get_duration_data(20200413, 20200420)
e = time.time()
print(e-s)

189.81447792053223


In [95]:
pd.DataFrame(data_7day.groupby('pfid')['ts'].transform('sum'))

,ts
0,191577
1,556952
2,1695198
3,61744
4,520105
...,...
194842,199871
194843,1024735
194844,491432
194845,840704
